In [1]:
from dataset.Dataset import T2WDataset
from network.cnn import Simple3DCNN
from network.rad_image_net import Backbone, Classifier
from torch.utils.data import DataLoader
import torch
from torch import nn

In [2]:
# Split dataset
# IMPORTANT: Use the same seed for the random split as during training
t2w_dataset = T2WDataset()
dataset_train, dataset_val = torch.utils.data.random_split(t2w_dataset, [int(0.7*len(t2w_dataset)), len(t2w_dataset)-int(0.7*len(t2w_dataset))], generator=torch.Generator().manual_seed(10))

In [8]:
from training import load_medicalnet_classifier

"""
# load radimagenet
backbone = Backbone()
num_classes = 5
classifier = Classifier(num_class=num_classes)
backbone.load_state_dict(torch.load("../data/RadImageNet/pretrained/RadImageNet_pytorch/ResNet50.pt"))
model = nn.Sequential(backbone, classifier)
"""

# load MedicalNet
model = load_medicalnet_classifier(num_classes=5, device=torch.device('cpu'))

model.load_state_dict(torch.load("model_checkpoints/medicalnet03.pth"))

/mnt/ADLM/users/weiserf/Documents/prostate-cancer-aggressiveness-prediction/src/../data/MedicalNet/models/resnet.py:174: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


param name: model.conv1.weight
param name: model.bn1.weight
param name: model.bn1.bias
param name: model.layer1.0.conv1.weight
param name: model.layer1.0.bn1.weight
param name: model.layer1.0.bn1.bias
param name: model.layer1.0.conv2.weight
param name: model.layer1.0.bn2.weight
param name: model.layer1.0.bn2.bias
param name: model.layer1.1.conv1.weight
param name: model.layer1.1.bn1.weight
param name: model.layer1.1.bn1.bias
param name: model.layer1.1.conv2.weight
param name: model.layer1.1.bn2.weight
param name: model.layer1.1.bn2.bias
param name: model.layer1.2.conv1.weight
param name: model.layer1.2.bn1.weight
param name: model.layer1.2.bn1.bias
param name: model.layer1.2.conv2.weight
param name: model.layer1.2.bn2.weight
param name: model.layer1.2.bn2.bias
param name: model.layer2.0.conv1.weight
param name: model.layer2.0.bn1.weight
param name: model.layer2.0.bn1.bias
param name: model.layer2.0.conv2.weight
param name: model.layer2.0.bn2.weight
param name: model.layer2.0.bn2.bias
p

<All keys matched successfully>

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda:1')
else:
    device = torch.device('cpu')

model.to(device)

MedicalNetClassifier(
  (model): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNor

In [10]:
def evaluate_model(model, dataset, device, n_classes=5):
    model.eval()

    # Create a DataLoader
    dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

    total_correct = 0
    total_samples = 0

    correct_per_class = torch.zeros(n_classes)
    total_per_class = torch.zeros(n_classes)

    # Iterate over DataLoader
    for batch in dataloader:
        
        data, labels = batch["image"], batch["label"]
        data, labels = data.to(device), labels.to(device)
        #data = data.unsqueeze(0)
        outputs = model(data)
        _, predicted_labels = torch.max(outputs, dim=1)

        # Update overall accuracy
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += labels.size(0)

        # Update per-class accuracy
        for i in range(n_classes):
            correct_per_class[i] += torch.sum((predicted_labels == i) & (labels == i)).item()
            total_per_class[i] += torch.sum(labels == i).item()
        
    overall_accuracy = total_correct / total_samples
    per_class_accuracy = correct_per_class / total_per_class
    per_class_accuracy[total_per_class == 0] = float('nan')  # Handle division by zero

    return overall_accuracy, per_class_accuracy

Print training set statistics

In [11]:
overall_accuracy, per_class_accuracy = evaluate_model(model, dataset_train, device)
print("Overall training set accuracy:", overall_accuracy)
print("Per-class training set accuracy:", per_class_accuracy)

Overall training set accuracy: 0.2
Per-class training set accuracy: tensor([0.0000, 0.0000, 0.0526, 0.7059, 0.0000])


Print validation set statistics

In [13]:
overall_accuracy, per_class_accuracy = evaluate_model(model, dataset_val, device)
print("Overall validation set accuracy:", overall_accuracy)
print("Per-class validation set accuracy:", per_class_accuracy)

Overall validation set accuracy: 0.20689655172413793
Per-class validation set accuracy: tensor([0.0000, 0.0000, 0.4000, 0.5000, 0.4000])
